## Q1. How does bagging reduce overfitting in decision trees?

# Q2. What are the advantages and disadvantages of using different types of base learners in bagging?

##  Q3. How does the choice of base learner affect the bias-variance tradeoff in bagging?

## Q4. Can bagging be used for both classification and regression tasks? How does it differ in each case?

# Q5. What is the role of ensemble size in bagging? How many models should be included in the ensemble?

## Q6. Can you provide an example of a real-world application of bagging in machine learning?

In [6]:
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [8]:
df=sns.load_dataset('tips')

In [10]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [38]:
df.time.value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [40]:
df['time']=df['time'].map({'Dinner':1 ,'Lunch':2})

In [42]:
x=df.drop('time',axis=1)
y=df['time']

In [44]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.impute import SimpleImputer

In [48]:
num_cols=['total_bill', 'tip', 'size']
num_cols

['total_bill', 'tip', 'size']

In [50]:
cate_cols=x.select_dtypes(include='category').columns
cate_cols

Index(['sex', 'smoker', 'day'], dtype='object')

In [52]:
num_pipline=Pipeline(
    steps=[
        ('impute',SimpleImputer(strategy='median')),
        ('scale',StandardScaler())
	 ]
)

cate_pipline=Pipeline(
    steps=[
        
        ('impute',SimpleImputer(strategy='most_frequent')),
        ('encode',OneHotEncoder()),
 			
	 ]
)

obj=ColumnTransformer(
    [
        ('num_cols',num_pipline,num_cols),
        ('cate_cols',cate_pipline,cate_cols)
	 ]
)

In [54]:
x_train_sc=obj.fit_transform(x_train)
x_test_sc=obj.transform(x_test)

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [74]:
models=[
                ('LogisticRegression',LogisticRegression()),
                ('KNeighborsClassifier', KNeighborsClassifier()),
                ('DesisionTree',DecisionTreeClassifier()),
                ('RandomForest',RandomForestClassifier())
                
]

In [60]:
from sklearn.metrics import r2_score

In [62]:
from sklearn.ensemble import BaggingClassifier

In [82]:
for name, clf in models:
    model=BaggingClassifier(base_estimator=clf,n_estimators=50,random_state=45)
    model.fit(x_train_sc,y_train)
    y_pred=model.predict(x_test_sc)
    
    acc=r2_score(y_test,y_pred)
    
    print(f'{model} accuracy {acc}')
    print('==============================')

BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=50,
                  random_state=45) accuracy 0.4916666666666667
BaggingClassifier(base_estimator=KNeighborsClassifier(), n_estimators=50,
                  random_state=45) accuracy 0.4916666666666667
BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=50,
                  random_state=45) accuracy 0.4916666666666667
BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=50,
                  random_state=45) accuracy 0.4916666666666667
